# Setup

In [371]:
import random
import string

from pylibdmtx.pylibdmtx import encode

from torchvision.transforms import v2
from torchvision.transforms.v2.functional import adjust_brightness

import numpy as np

# Setting Parameters

To synthesize the dataset the following parameters and amounts will be used:

- 20 randomized strings in following letter (L) number (N) format:
    - NLNLNNNNNNN + NNNN
    - \+ NNNN denotes the last 4 numbers in a serial number which increment by 1. Therefore this will range from 0000 to 0100 to mimic the real-world data more closely.
- 20 configurations of shape transformations of the following types:
    - random horizontal flip
    - random vertical flip
    - random rotation
    - random affine
    - random perspective
- 5 different scalings between 0-1
- 20 configurations of color transformations of the following types:
    - color jitter
    - random photometric distort
    - random grayscale
    - gaussian blur
    - gaussian noise
    - random invert
    - random adjust sharpness

This will lead to $20*20*5*20*2 = 80,000$ images being generated (40,000 ground truth and 40,000 noisy data).

## Helpers

These helpers do each individual part of the pipeline

In [413]:
def gen_string():
    '''
    Generates a serial number to encode
    
    Serial numbers are:
    - 11 characters long
    - Index 0, 2, 4, 5, 6, 7, 8, 9, 10 are random digits
    - Index 1 and 3 are uppercase letters
    - Index 11, 12, 13, 14 are an incremental number starting from 0001

    Example serial number: 4 L 4 N 0418028 0001
    '''
    to_encode = ''

    # first 11 indexes
    for j in range(11):
        # 1 and 3 are uppercase
        if j in [1, 3]:
            to_encode += random.choice(string.ascii_uppercase)
        else:
            to_encode += str(random.randrange(0, 10))
    
    # last 4 indexes
    end = str(random.randrange(1, 99))
    if len(end) == 1:
        end = '0' + end
    elif len(end) == 2:
        end = '00' + end
    else:
        end = '000' + end
    to_encode += end

    return to_encode

def encode_image(to_encode):
    '''Creates a PIL image containing DMC encoding of given string'''
    encoded = encode(to_encode.encode('utf8'))
    img = Image.frombytes('RGB', (encoded.width, encoded.height), encoded.pixels)
    img = img.crop((10, 10, img.width-10, img.height-10)) # crop image to remove white borders
    img = img.resize((img.width*10, img.height*10), Image.BILINEAR) # upscale image
    return img

def shape_transform(img):
    '''Applies random shape transformations to image'''
    transforms = v2.Compose([
        v2.Pad(1500, fill=255, padding_mode='constant'),
        v2.RandomHorizontalFlip(),
        v2.RandomVerticalFlip(),
        v2.RandomRotation(random.randrange(0, 360), fill=255, interpolation=Image.BILINEAR),
        v2.RandomAffine(degrees=0,
                        scale=(0.5, 1.5), # randomly scale image size between 0.5 and 1.5
                        # random "squish" on x and y axis
                        shear=(-random.randrange(10,20), random.randrange(10,20), -random.randrange(10,20), random.randrange(10,20)),
                        fill=255,
                        interpolation=Image.BILINEAR,
                        ),
        v2.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=Image.BILINEAR, fill=255),
    ])

    img = transforms(img)
    return img

def brightness_transform(img, increment):
    '''
    Applies random brightness transformations to image
    Incremenets by 1 initially to avoid 0 pixel values
    Converts to int16 to avoid overflow (making white pixels black)
    '''
    # Incremement by 1 as 0 pixel values do not get scaled
    img = np.array(img, dtype=np.int16)
    img = np.clip(img + 1, 0, 255)
    img = img.astype(np.uint8)
    img = Image.fromarray(img)
    return adjust_brightness(img, 250.0)

In [ ]:
def gen_params(N_strings, N_shapes, N_scalings, N_colors):
    '''Generates parameters for N images to feed to the image generator'''
    # generate N_strings DMCs
    imgs = []
    for _ in range(N_strings):
        to_encode = gen_string()
        img = encode_image(to_encode)
        imgs.append(img)
        break

    # generate N_shapes shape transformed DMCs of each DMC
    for idx, img in enumerate(imgs):
        for _ in range(N_shapes):
            img = shape_transform(img)
            imgs[idx] = img
            break
        break
    ground_truth = imgs.copy()

    # generate N_scalings scaled DMCs of each DMC
    increments = [0, 25, 50, 75, 100, 125, 150, ]
    for img in imgs:
        for _ in range(N_scalings):
            img = brightness_transform(img, increment=)
            img.show()
            break
        break

    # generate N_colors colored DMCs of each DMC
    for img in imgs:
        for _ in range(N_colors):
            pass

    return

def gen_save_imgs(params, path):
    '''Given parameters and path to save, generates and saves both a ground truth image and a noisy image'''
    return

gen_params(
    N_strings=20,
    N_shapes=20,
    N_scalings=5,
    N_colors=20
)

TypeError: brightness_transform() missing 1 required positional argument: 'increment'